### ***Multilayer Perceptron and Backpropagation***

In [269]:
''' Import all important library '''
import torch
import torch.nn as nn
import torch.optim as gredient
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [270]:
Device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device Name is {Device}')

Device Name is cpu


### ***Data Pipeline***

In [271]:
Transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.1307),
        std=(0.3181)
    )
])
'''
Compose = all number convert between (0 - 1)[pixel_num / 255]
standard division
'''

'\nCompose = all number convert between (0 - 1)[pixel_num / 255]\nstandard division\n'

In [272]:
Train_Dataset = datasets.MNIST(
    root='/Users/mahadiur/Desktop/Deep Learning Forward and Backpropagation/Data',
    train=True,
    download=True,
    transform=Transformation
)

Test_Dataset = datasets.MNIST(
    root='/Users/mahadiur/Desktop/Deep Learning Forward and Backpropagation/Data',
    train=False,
    download=True,
    transform=Transformation
)

In [273]:
print(type(Train_Dataset))

<class 'torchvision.datasets.mnist.MNIST'>


### ***Dataloader***

In [274]:
Train_Dataloader = DataLoader(
    Train_Dataset,
    batch_size=32,
    shuffle=True
)

Test_Dataloader = DataLoader(
    Test_Dataset,
    batch_size=32,
    shuffle=False
)

### ***Perceptron***

In [275]:
class Perceptron(nn.Module):
    def __init__(self, input_size):
        super(Perceptron, self).__init__()
        self.weights = nn.Parameter(torch.randn(input_size))
        self.bias = nn.Parameter(torch.randn(1))
    
    def forward(self, x):
        sumation = x @ self.weights + self.bias
        return sumation

In [276]:
''' Testing '''
input_size = 28 * 28
shuffle = torch.randn(1, input_size)

perceptron = Perceptron(
    input_size=input_size
)

output = perceptron(shuffle)
print(output)

tensor([-46.2171], grad_fn=<AddBackward0>)


In [277]:
'''
Some import term for image classification:
- Batch Size (8, 32, 64, 128)
- epoch
- Step 
MNIST Dataset :
- Grayscale image 
- pixel(28 * 28)
- 60000 images in MNIST dataset
'''

'\nSome import term for image classification:\n- Batch Size (8, 32, 64, 128)\n- epoch\n- Step \nMNIST Dataset :\n- Grayscale image \n- pixel(28 * 28)\n- 60000 images in MNIST dataset\n'

### ***Activation Function***

In [278]:
'''
Activation Function: RELU (Rectified Linear Unit)
Function: max(0, z)  
'''
class RELU(nn.Module):
    def __init__(self):
        super(RELU, self).__init__()
    def forward(self, x):
        return torch.maximum(torch.tensor(0.0), x)

In [279]:
relu = RELU()
output_relu = relu(output)
print(output_relu)

tensor([0.], grad_fn=<MaximumBackward0>)


### ***Perceptron Hidden Layer***

In [280]:
''' Perceptron Hidden Layer '''
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.perceptron = nn.ModuleList([
            Perceptron(input_size) for _ in range(output_size)
        ])
    def forward(self, x):
        outputs = [
            perceptron(x) for perceptron in self.perceptron
        ]
        final_output =torch.stack(outputs, dim=1)
        return final_output


# Testing
linear = Linear(input_size,10) 
output = linear(shuffle)
print(output)
print(output.shape)

tensor([[-46.4985,  -3.3348,  -9.3832, -70.5822,  29.9930,  38.8538,  10.2868,
          30.2287,  25.5940,  32.7509]], grad_fn=<StackBackward0>)
torch.Size([1, 10])


### ***Digit Classifiar betweet (1, 10)***

In [281]:
class Digitclassifiar(nn.Module):
    def __init__(self, input_size=28 * 28, output_size=10):
        super().__init__()
        self.layer1 = Linear(input_size, 256 )
        self.layer2 = Linear(256, 128)
        self.layer3 = Linear(128, 64)
        self.layer4 = Linear(64, output_size)
        self.relu = RELU()
    
    def forward(self, x):
        x = x.view(-1, input_size) # [1 ... 784]
        x = self.layer1(x) # [1 ... 256]
        x = self.relu(x) # activation function
        x = self.layer2(x) # [1...128]
        x = self.relu(x)
        x = self.layer3(x) # [1...64]
        x = self.relu(x)
        x = self.layer4(x) # [1...10]
        return x

In [282]:
Model = Digitclassifiar(input_size=28 * 28, output_size=10).to(Device)
output = Model(shuffle)
print(output.shape)
print(output)

torch.Size([1, 10])
tensor([[-24648.2676,  -7700.8711,  -6684.8853,   3203.1455,   5297.8779,
           7337.7197,   8830.7939,  14088.4092,  -1146.4935, -10406.6758]],
       grad_fn=<StackBackward0>)


### ***Run epochs***

In [283]:
optimizer = gredient.Adam(Model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()
epochs = 5

for epoch in range(epochs):
    Model.train()
    current_loss = 0.0
    for batch_num, (data, target) in enumerate(Train_Dataloader):
        data, target = data.to(Device), target.to(Device)
        outputs = Model(data)
        loss = loss_function(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        current_loss += loss.item()
        if epoch % 100 == 0:
            print(f'Epoch {epoch + 1} / {epochs} Step [{batch_num + 1 }/ {len(Train_Dataloader)}] and Loss {loss.item():3f}')


Epoch 1 / 5 Step [1/ 1875] and Loss 15599.382812
Epoch 1 / 5 Step [2/ 1875] and Loss 19830.091797
Epoch 1 / 5 Step [3/ 1875] and Loss 16532.462891
Epoch 1 / 5 Step [4/ 1875] and Loss 17218.728516
Epoch 1 / 5 Step [5/ 1875] and Loss 17456.179688
Epoch 1 / 5 Step [6/ 1875] and Loss 13997.030273
Epoch 1 / 5 Step [7/ 1875] and Loss 12747.017578
Epoch 1 / 5 Step [8/ 1875] and Loss 15200.763672
Epoch 1 / 5 Step [9/ 1875] and Loss 10721.020508
Epoch 1 / 5 Step [10/ 1875] and Loss 10920.533203
Epoch 1 / 5 Step [11/ 1875] and Loss 12241.839844
Epoch 1 / 5 Step [12/ 1875] and Loss 11333.862305
Epoch 1 / 5 Step [13/ 1875] and Loss 9360.697266
Epoch 1 / 5 Step [14/ 1875] and Loss 10270.335938
Epoch 1 / 5 Step [15/ 1875] and Loss 10791.062500
Epoch 1 / 5 Step [16/ 1875] and Loss 10230.234375
Epoch 1 / 5 Step [17/ 1875] and Loss 7661.676270
Epoch 1 / 5 Step [18/ 1875] and Loss 9968.195312
Epoch 1 / 5 Step [19/ 1875] and Loss 8357.970703
Epoch 1 / 5 Step [20/ 1875] and Loss 10869.579102
Epoch 1 / 5 S

In [284]:
print(loss)

tensor(65.7611, grad_fn=<NllLossBackward0>)
